In [3]:
import importlib
import pandas as pd
import odqa_encoder
import odqa_milvus
import odqa_mysql
from answer_extractor import answer_extract


In [1]:
question = ["மனித உடலில் எத்தனை எலும்புகள் உள்ளன?"]


In [5]:
question_emb = odqa_encoder.encode(question)

similar_ids = odqa_milvus.find_similar(question_emb)
sim_ids = similar_ids[0].ids
retrieved_context = odqa_mysql.extract_context(sim_ids[0])

In [6]:
QA_input = {
    'question': question,
    'context': retrieved_context
}
res = answer_extract(QA_input)
res

[{'score': 3.443395598878851e-06,
  'start': 507,
  'end': 514,
  'answer': ' நான்கு'}]

In [2]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
connections.connect()
TABLE_NAME = 'question_answering'

In [3]:
field1 = FieldSchema(name="ind", dtype=DataType.INT64, descrition="int64", is_primary=True)
field2 = FieldSchema(name="id", dtype=DataType.INT64, descrition="int64", is_primary=False)
field3 = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, descrition="float vector",dim=768, is_primary=False)
schema = CollectionSchema(fields=[field1, field2, field3], description="collection description")
collection = Collection(name=TABLE_NAME, schema=schema)
default_index = {"index_type": "IVF_FLAT", "metric_type": 'IP', "params": {"nlist": 200}}
collection.create_index(field_name="embedding", index_params=default_index)

Status(code=0, message='')

In [4]:
search_params = {"metric_type": "IP", "params": {"nprobe": 10}}


In [5]:
def find_similar(emb):
    collection.load()
    return collection.search(
	data=emb, 
	anns_field="embedding", 
	param=search_params, 
	limit=10, 
	expr=None,
	consistency_level="Strong"
)